In [1]:
import csv
import pandas as pd
from clarke_wright import clarke_wright
from clarke_wright import vehicle
from clarke_wright import customer
from clarke_wright import calculate_cost

In [2]:
# REAL INSTANCES

# Initialize an empty list to store each row as a dictionary
data = []

# Read the CSV file
with open('instances.csv', mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        # Convert necessary fields from strings to actual data types
        row['q'] = eval(row['q'])
        row['e'] = eval(row['e'])
        row['s'] = eval(row['s'])
        row['l'] = eval(row['l'])
        row['d'] = eval(row['d'])
        row['t'] = eval(row['t'])
        row['a'] = eval(row['a'])
        row['cf'] = eval(row['cf'])
        row['R'] = eval(row['R'])
        
        # Append the row dictionary to the data list
        data.append(row)

In [3]:
# OPTIMUM VALUES

# Initialize an empty list to store each row as a dictionary
optimum_data = []

# Read the CSV file
with open('optimum.csv', mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        row['optimum_cost'] = float(row['optimum_cost'])
        row['true_cost'] = float(row['true_cost'])
        # Append the row dictionary to the data list
        optimum_data.append(row)

In [4]:
columns = ['Cost', 'Optimum Cost', 'Gap %']
df = pd.DataFrame(columns=columns)

In [5]:
def check_routing(routes, n):
    # Input: list of routes and number of customers
    # Output: whether all customers belong to at least one route

    routed = []
    for k in routes:
        route = k[0]
        for j in route:
            if not j in routed: routed.append(j)
    print(routed)
    if len(routed) < n + 1:    # The depot must be taken into account
        return False
    return True

In [6]:
check_routing([[[0,2,3,0],0],[[0,1,0],1]], 4)

[0, 2, 3, 1]


False

In [7]:
results = []

for i in range(len(data)):        # CHANGE IT FOR I IN RANGE TO WORK WITH BOTH OPTIMUM AND INSTANCES DATA
    # Get parameters from data
    row = data[i]
    n = int(row['n'])
    V = int(row['V'])
    q = row['q']
    e = row['e']
    s = row['s']
    l = row['l']
    d = row['d']
    t = row['t']
    a = row['a']
    cf = row['cf']
    R = row['R']

    print(n,V,q,e,s,l,d,t,a,cf,R)

    # Create a list of customers using the input data
    customers = [
        customer(id, q[id], e[id], s[id], l[id]) for id in range(n+1)
    ]

    # Create a list of vehicles using the input data
    vehicles = [
        vehicle(id, a[id], cf[id], R[id]) for id in range(V)
    ]

    # Run Clarke-Wright algorithm and calculate costs
    final_routes, f, wait = clarke_wright(customers, vehicles, d, t, R)
    print(f'final routes = {final_routes, f}')
    cost = calculate_cost(final_routes, d, vehicles)

    # Calculate optimum gap
    optimum_cost = optimum_data[i]['optimum_cost']
    gap = (cost - optimum_cost) / optimum_cost
    
    # Print row preview
    print(f'Cost = {cost}, Optimum Cost = {optimum_cost}, Gap %% = {round(gap*100,2)}%%, Routes = {final_routes}, f = {f}')
    print(check_routing(final_routes, n))
    print()

    # Append values to dataframe
    df.loc[len(df)] = [cost, optimum_cost, round(gap * 100, 2)]

 

5 2 [0, 14, 1.2, 2, 0.4, 10] [8.0, 8.0, 8.0, 8.0, 8.0, 8.0] [0, 1.5, 1.5, 1.5, 1.5, 1.5] [18.0, 18.0, 18.0, 18.0, 18.0, 18.0] [[0, 106, 116, 47, 57, 58], [106, 0, 21, 117, 127, 127], [116, 21, 0, 119, 128, 128], [47, 117, 119, 0, 11, 12], [57, 127, 128, 11, 0, 2], [58, 127, 128, 12, 2, 0]] [[0, 2.65, 2.9, 1.18, 1.43, 1.45], [2.65, 0, 0.53, 2.93, 3.18, 3.18], [2.9, 0.53, 0, 2.98, 3.2, 3.2], [1.18, 2.93, 2.98, 0, 0.28, 0.3], [1.43, 3.18, 3.2, 0.28, 0, 0.05], [1.45, 3.18, 3.2, 0.3, 0.05, 0]] [16.01, 22.02] [3.13, 2.57] [[1, 1, 1, 1, 1], [1, 1, 0, 0, 1]]
final routes = ([[[0, 3, 4, 5, 0], 0], [[0, 2, 1, 0], 1]], [[0.0, 0.0, 0.0, 1.0, 1.0, 1.0], [0.0, 1.0, 1.0, 0.0, 0.0, 0.0]])
Cost = 993.85, Optimum Cost = 994.0, Gap %% = -0.02%%, Routes = [[[0, 3, 4, 5, 0], 0], [[0, 2, 1, 0], 1]], f = [[0.0, 0.0, 0.0, 1.0, 1.0, 1.0], [0.0, 1.0, 1.0, 0.0, 0.0, 0.0]]
[0, 3, 4, 5, 2, 1]
True

5 3 [0, 14, 7.56, 0.8, 2, 6.78] [8.0, 8.0, 8.0, 8.0, 8.0, 8.0] [0, 1.5, 1.5, 1.5, 1.5, 1.5] [18.0, 18.0, 18.0, 18.0, 

In [8]:
# Add 'id' column starting from 1 to reflect instances id
df.insert(0, 'id', range(1, len(df) + 1))

In [9]:
df

,id,Cost,Optimum Cost,Gap %
0,1,993.85,994.0,-0.02
1,2,2027.26,1296.4,56.38
2,3,181.54,178.4,1.76
3,4,1021.50,935.2,9.23
4,5,2261.12,1915.6,18.04
5,6,1988.52,1253.2,58.68
6,7,1192.29,665.2,79.24
7,8,1217.35,1106.0,10.07
8,9,1902.82,1303.6,45.97
9,10,2689.96,1869.2,43.91
